In [1]:
from bert2head.model import BertCNN2HEAD, BertLinear2HEAD, BertCNN2HEAD_UIT
import torch



cuda


In [2]:
net = BertLinear2HEAD('vinai/phobert-base-v2')
net.load_state_dict(torch.load("models/linear/Best-bert-ever.pt"))


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
torch.save(net.BertModel.state_dict(),"models/finetuneBertsmart.pt")

In [4]:
import torch
s = torch.ones(32,6)

In [13]:
import numpy as np

In [54]:
a = np.array([32,14,67,3,6]).reshape(-1,1)

np.random.randint(0,a)

array([[ 4],
       [ 4],
       [18],
       [ 2],
       [ 4]])

In [55]:
a=[torch.Tensor([4,3]),torch.Tensor([30,50])]
b=torch.cat(a).view(2,2)

In [56]:
b[b !=3] =1

In [57]:
b

tensor([[1., 3.],
        [1., 1.]])

In [40]:
c=torch.Tensor([0,0]).type(torch.IntTensor).reshape(-1,1)
c

tensor([[0],
        [0]], dtype=torch.int32)

In [44]:
b[torch.arange(2)[:,None],c]=1

In [45]:
b

tensor([[ 1.,  3.],
        [ 1., 50.]])